# Tutorial 1: SFT with LoRA

• Find the  [Notebook](https://colab.research.google.com/github/towardsai/ragbook-notebooks/blob/main/notebooks/Chapter%2010%20-%20FineTuning_a_LLM_LIMA_CPU.ipynb)  for this section at  [towardsai.net/book](http://towardsai.net/book).

The upcoming section will demonstrate an example of fine-tuning a model using the LoRA approach on a small dataset. This technique allows for instruction tuning of large language models by applying low-rank adaptations to modify the model’s behavior without retraining its weights. This process is highly efficient and can be executed using a CPU on a Google Cloud instance. We will guide you through preparing the dataset and conducting the fine-tuning process using the Hugging Face library. This hands-on example will provide a practical understanding of enhancing model performance with minimal computational resources and dataset size.

While the Hugging Face library is used in this example, the following libraries provide a range of tools, optimizations, compatibility with various data types, resource efficiency, and user-friendly interfaces to support different tasks and hardware setups, enhancing the efficiency of the LLM fine-tuning process.

-   [PEFT Library](https://github.com/huggingface/peft): The Parameter-Efficient Fine-Tuning (PEFT) library enables the efficient adaptation of pre-trained language models to various downstream applications without fine-tuning all the model’s parameters. Methods like LoRA, Prefix Tuning, and P-tuning are part of PEFT.
-   [Lit-GPT](https://github.com/Lightning-AI/lit-gpt): Developed by LightningAI, Lit-GPT is also an open-source tool designed to streamline fine-tuning. It facilitates the application of techniques like LoRA without manual modifications to the core model architecture. Models such as  [Vicuna](https://lmsys.org/blog/2023-03-30-vicuna/),  [Pythia](https://www.eleuther.ai/papers-blog/pythia-a-suite-for-analyzing-large-language-modelsacross-training-and-scaling), and  [Falcon](https://falconllm.tii.ae/)  are available. Lit-GPT allows applying specific configurations to different weight matrices and offers adjustable precision settings to manage memory usage effectively.

We also recommend using virtual machines on systems like GCP, which is good practice in the industry and might be better than using your current system in most cases. To do so, log in to the Google Cloud Platform account and create a  [Compute Engine](https://cloud.google.com/compute)  instance. You can select from various  [machine types](https://cloud.google.com/compute/docs/cpu-platforms). Cloud GPUs are a popular option for many deep learning applications, but CPUs can also effectively optimize LLMs. For this example, we train the model using a CPU.

Whatever type of machine you choose to use, if you encounter an out-of-memory error, try reducing parameters such as `batch_size or seq_length`.

>⚠️**Note:**  Costs are associated with starting up virtual machines. The total cost will depend on the type of machine and how long it is running. Regularly check your costs in the billing section of GCP and turn off your virtual machines when you’re not using them.

>💡If you want to run the code in the section without spending much money, you can perform a few iterations of training on your virtual machine and then stop it.

Install the dependences:

    !pip install -q transformers==4.32.0 deeplake[enterprise]==3.7.1 trl==0.6.0 peft==0.5.0 wandb==0.15.8


### Load API Keys

In [ ]:
from fine_tuning_custom_utils.helper import get_openai_api_key, get_activeloop_api_key
OPENAI_API_KEY = get_openai_api_key()
ACTIVELOOP_API_KEY = get_activeloop_api_key()

## Loading the Dataset

The quality of a model’s output is directly tied to the quality of the data used for training. Start with a well-planned dataset, whether open-source or custom-created. We will use the dataset from the “[LIMA: Less Is More for Alignment](https://arxiv.org/pdf/2305.11206.pdf)” paper. This research suggests that a small, meticulously selected dataset of a thousand samples could replace the RLHF strategy. This research is publicly available under a non-commercial use license.

The Deep Lake database infrastructure by Activeloop enables dataset streaming, eliminating the need to download and load the dataset into memory.

The code will create a loader object for the training and test sets:

In [ ]:
import deeplake

# Connect to the training and testing datasets
ds = deeplake.load('hub://genai360/GAIR-lima-train-set')
ds_test = deeplake.load('hub://genai360/GAIR-lima-test-set')

print(ds)

    Dataset(path='hub://genai360/GAIR-lima-train-set', read_only=True, tensors=['answer', 'question', 'source'])

The pre-trained tokenizer object for the [Open Pre-trained Transformer (OPT)](https://arxiv.org/abs/2205.01068) LLM is initially loaded using the transformers library, and the model is loaded later. We chose OPT for its open availability and relatively moderate parameter count. However, the code in this section is versatile and can be applied to other models. For example, you could use `meta-llama/Llama-2-7b-chat-hf` for [Llama 2](https://huggingface.co/meta-llama/Llama-2-7b-chat-hf).

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("facebook/opt-1.3b")

The `prepare_sample_text processes` a row of data stored in Deep Lake, organizing it to start with a question and follow with an answer, separated by two newlines. Defining a formatting function helps the model learn the template and recognize that a prompt beginning with the `question` keyword should typically be completed with an answer.

In [ ]:
def prepare_sample_text(example):
    """Prepare the text from a sample of the dataset."""
    text = f"""Question: {example['question'].text()}\n\nAnswer: {example['answer'].text()}"""

    return text

The Hugging Face  [TRL library](https://github.com/huggingface/trl)  integrates the SFT method and allows the integration of LoRA configurations, simplifying the implementation.

To prepare the training and evaluation datasets for fine-tuning, we use the `ConstantLengthDataset` object from the TLR library. This object requires a tokenizer, the Deep Lake dataset, and a formatting function, `prepare_sample_text`, which processes the data into the required format.

The `seq_length` parameter determines the maximum sequence length used during training and evaluation, aligning it with the model’s configuration. While the value can be set as high as 2048 for models that can handle long contexts, we’ve selected 1024 in this example to optimize memory usage. You may increase this value depending on your dataset’s text length and available resources.

Additionally, setting `infinite=True` ensures that once all data points are exhausted, the dataset iterator automatically restarts, making it ideal for continual training scenarios.

In [ ]:
from trl.trainer import ConstantLengthDataset

train_dataset = ConstantLengthDataset(
    tokenizer,
    ds,
    formatting_func=prepare_sample_text,
    infinite=True,
    seq_length=1024
)

eval_dataset = ConstantLengthDataset(
    tokenizer,
    ds_test,
    formatting_func=prepare_sample_text,
    seq_length=1024
)

# Show one sample from train set
iterator = iter(train_dataset)
sample = next(iterator)
print(sample)

    {'input_ids': tensor([ 2, 45641, 35, ..., 48443, 2517, 742]), 'labels': tensor([ 2, 45641, 35, ..., 48443, 2517, 742])}

The output shows that the `ConstantLengthDataset` class took care of all the necessary steps to prepare our dataset.

>💡If you use the iterator to print a sample from the dataset, execute the following code to reset the iterator pointer: train_dataset.start_iteration = 0.

## Configuring LoRA and Training Hyperparameters

Set the LoRA configuration using the PEFT library. The variable r indicates the dimension of matrices, where lower values mean fewer trainable parameters. `lora_alpha` acts as the scaling factor. The `bias` specifies which bias parameters should be trained with options like `none`, `all`, and `lora_only`.


In [ ]:
from peft import LoraConfig

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

Next, set the `TrainingArguments`. A higher learning rate combined with increased weight decay can enhance the fine-tuning performance. Additionally, it is good to use `bf16=True` as it can reduce memory usage during fine-tuning.

We also set up the  [Weights & Biases](https://wandb.ai/site)’  tracking. This platform monitors and records every aspect of the process, including metrics like loss, accuracy, and gradients in real-time. By integrating it with the training script, you can visualize training curves, compare runs, and debug issues easily. It provides insights into the training process, helping you optimize hyperparameters and detect overfitting or underfitting early on.

To integrate this tool, install the package by running `pip install wandb` and use the `wandb` parameter in the `report_to` argument to manage the logging process effectively:

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./OPT-fine_tuned-LIMA-CPU",
    dataloader_drop_last=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=10,
    logging_steps=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=1e-4,
    lr_scheduler_type="cosine",
    warmup_steps=10,
    gradient_accumulation_steps=1,
    bf16=True,
    weight_decay=0.05,
    run_name="OPT-fine_tuned-LIMA-CPU",
    report_to="wandb",
)

Load the pre-trained `facebook/opt-1.3b` model. The model will be loaded using the transformers library.

In [ ]:
from transformers import AutoModelForCausalLM
import torch

model = AutoModelForCausalLM.from_pretrained("facebook/opt-1.3b",
torch_dtype=torch.bfloat16)

The following code will loop over the model parameters, converting the data type of specified layers (such as LayerNorm and the final language modeling head) to a 32-bit format. This improves the stability of fine-tuning.

In [ ]:
import torch.nn as nn

for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

Use the `SFTTrainer` class to connect all components. The model, training arguments, training dataset, and LoRA configuration are needed to build the trainer object. The `packing` option indicates that we previously packed samples together using the `ConstantLengthDataset` class:

In [ ]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    peft_config=lora_config,
    packing=True,
)

So, why did we use LoRA? Let’s see how it helped by writing a simple function that computes the number of available parameters in the model and compares it to the trainable parameters. The trainable parameters are those that LoRA added to the underlying model.

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"""trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"""
    )

print(print_trainable_parameters(trainer.model))

    trainable params: 3145728 || all params: 1318903808 || trainable%: 0.23851079820371554

The trainable parameters are limited to 3 million. Only 0.2% of the total parameters are updated leveraging LoRA. It drastically minimizes the amount of RAM required.

The trainer object is now ready to begin the fine-tuning cycle by invoking the `.train() method`:

In [ ]:
print("Training...")

In [ ]:
trainer.train()

💡You can access the [OPT fine-tuned LIMA checkpoint on CPU](https://github.com/towardsai/rag-ebook-files/blob/main/OPT-fine_tuned-LIMA-CPU.zip) at [towardsai.net/book](http://towardsai.net/book). Additionally, find more information on [the Weights & Biases project page](https://wandb.ai/ala_/GenAI360/runs/uhe0kbku?nw=nwuserala_) on the fine-tuning process at [towardsai.net/book.](http://towardsai.net/book)

## Merging the LoRA and OPT parameters

The final step is to merge the base model with the trained LoRA parameters, resulting in a standalone model.

If operating in a new environment, load the base OPT-1.3B model:

In [ ]:
from transformers import AutoModelForCausalLM
import torch

model = AutoModelForCausalLM.from_pretrained(
  "facebook/opt-1.3b", return_dict=True, torch_dtype=torch.bfloat16
)

Use `PeftModel` to load the fine-tuned model by specifying the checkpoint path:

In [ ]:
from peft import PeftModel

# Load the Lora model
model = PeftModel.from_pretrained(
  model,
  "./OPT-fine_tuned-LIMA-CPU/<desired_checkpoint>/"
)
model.eval()

    PeftModelForCausalLM(  
      (base_model): LoraModel(  
        (model): OPTForCausalLM(
             ......

Combine the base model and LoRA layers using the PEFT model’s `.merge_and_unload()` method. The weights can be saved to disk with the `.save_pretrained()` method.

In [ ]:
model = model.merge_and_unload()

model.save_pretrained("./OPT-fine_tuned-LIMA/merged")

💡Note that the base model employed in this section is relatively small and has limited capabilities compared to state-of-the-art models, such as ChatGPT. The process from this section can be easily applied to train larger LLMs.

## Inference

The performance of the fine-tuned model can be estimated using a variety of prompts. The following code shows how to use Hugging Face’s `.generate()` method for straightforward interactions with the model. Several other arguments and decoding strategies are available that can improve text generation quality. You can explore these techniques further in the [blog post](https://huggingface.co/blog/how-to-generate) by Hugging Face (available at [towardsai.net/book](http://towardsai.net/book)).

In [ ]:
inputs = tokenizer("Question: Write a recipe with chicken.\n\n Answer: ",
return_tensors="pt")

generation_output = model.generate(**inputs,
                                   return_dict_in_generate=True,
                                   output_scores=True,
                                   max_length=256,
                                   num_beams=1,
                                   do_sample=True,
                                   repetition_penalty=1.5,
                                   length_penalty=2.)

print(tokenizer.decode(generation_output['sequences'][0]))

    Question: Write a recipe with chicken.\n\n Answer: \n* Chicken and rice is one of the most popular meals in China, especially during Chinese New Year celebrations when it's served as an appetizer or main course for dinner parties (or just to eat by yourself). It can be made from scratch using fresh ingredients like meatballs/chicken breasts if you have them on hand but otherwise use frozen ones that are already cooked so they don't need any additional cooking time before serving. You could also substitute some vegetables instead such as broccoli florets which would make this dish even more delicious! If your family doesn’t know how to cook well then I suggest making these recipes ahead of time because once done all you really do is reheat until hot again :)\n## Make homemade marinade\n1) Combine 1 tablespoon soy sauce, 2 tablespoons sesame oil, 3 teaspoons sugar, 4 cloves garlic minced into small pieces, 6-8 green onions chopped finely, 5 cups water, salt & pepper to taste, about 8 ounces boneless skinless chicken breast fillets cut up fine enough not to stick together while being mixed thoroughly - no bones needed here since there will only ever be two servings per person), ½ cup cornstarch dissolved in ¼...

Our experiment presented the same prompt to the original base model and the fine-tuned version. This evaluation aimed to compare each model’s ability to follow instructions accurately. Find the list of prompts with corresponding answers below.

1. Create a marketing plan for a coffee shop.

Vanilla Model Output:

    "</s>Question: Create a marketing plan for a coffee shop.\n\n Answer: "Do not make it more expensive than your competition, unless you wish to lose business." \nIs that even possible? I haven\'t seen the competition in my store and we\'ve only been here 2 years (although they are new). We keep ourselves busy just like everyone else there but if people can afford Starbucks at $4-5/person then wouldn\'t the market become saturated fast enough anyway no matter what brand or flavor you chose?\n"It\'s hard out\ 'un Texas...but somebody get me some money!" The quote came from a song by Celine Dion!</s>"

LoRA Model Output:

    "</s>Question: Create a marketing plan for a coffee shop.\n\n Answer: \n## Plan the business model and pricing strategy to make money from your cafe or restaurant\n1) Set up an online store with all of its products on it, including cups/tumblers etc., so that customers can order them directly through their website without having to go into any physical stores (e-commerce). This will help you get more sales as will since people are less likely to visit brick & mortar shops if they have no way to buy what is being sold there in person! You could also set this up by using Amazon\'s FBA program which allows sellers like yourself who don\'t want to deal with inventory management themselves but still need some sort of distribution channel available where orders come straight off the shelf instead of going out to be picked at warehouses before shipping – just Google "Amazon fulfillment center" :). If you\'re not comfortable doing something similar then consider setting one up via Shopify too; ..."


2. Why does it rain? Explain your answer.

Vanilla Model Output:

    "</s>Question: Why does it rain? Explain your answer.\n\n Answer: The moon’s gravity is a lot less than earths and so the sky never really gets dark, but rather stays mostly clear (which makes things easier because it can be difficult to see stars from below). Even when there are clouds in sight of another planet like Mars or Jupiter then you will still have plenty if light on that other planets surface regardless due to its size relative star density.\nWhy do cows breathe air too though!? My Knowledge might be off base here…..but seriously why??? Is this how animals die…by suffocation?? Would an animal take such risks without having been programmed with those behaviours for survival?! Does science just not care about life anymore!!!!!????\nWell they’re mammals after all!! They evolved *just* like us! We don’t understand them very well now :( I would wager most humans didn’t even know we had one until fairly recent times either :P Though obviously I’m sure some people knew somethi…’

LoRA Model Output:

    "</s>Question: Why does it rain? Explain your answer.\n\n Answer: \n* The Earth\'s atmosphere is a giant, complex system that has many different layers and processes at work to keep the planet wet (and dry). Rain falls on land because of gravity waves from other planets in our solar system or even distant galaxies; these gravitational forces are so strong they can push water molecules into clouds where precipitation occurs. Water droplets then fall as snowflakes onto surfaces like mountains, which act much more slowly than air currents do but still move enough for them all to be frozen solid before falling back down again when spring arrives. This process repeats itself over time until there isn\'t any moisture left anywhere except under ice sheets covering Antarctica. When this happens, we call winter “the end”-— no longer will you see anything growing outside during summer months! But if you look closely, some plants grow underground instead—they\'re called mosses since their roots ..."

3. What’s the Italian translation of the word “house”?

Vanilla Model Output:

    "</s>Question: What\'s the Italian translation of the word \'house\'?.\n\n Answer: --It means nothing, really! It is just someone who owns houses. For example you can say "the house where I live" or something like that»\nThe meaning should be a home for people in Italy as well? :P (Italian) Edit: not sure it quite fits though... ;)</s>"

LoRA Model Output:

    "</s>Question: What's the Italian translation of the word \'house\'?.\n\n Answer: \n* "House" is a noun, which means an enclosed space or building in English (and other languages). The verb form for house can be either singular ("I live at home") and plural (*) ("We have houses"), but it usually refers to one person living there alone; e.g., I am my own landlord because we don\'t share any property with eachother. In some cases you may also see * (noun), like when someone says they are their parents\' only child - that would mean both parentage as well as residence together on this planet! For example, if your mother lives next door while her father works out-of town all day long she could say something along these lines about herself being his sole tenant/owner without having to specify who owns what piece of real estate between them...\nIn general though, people tend not to use words such as apartment / flat etc.; instead using terms more commonly used by Americans including single family homes & multi unit dwellings. This includes

The results show the limitations and strengths of both models. The LoRA-tuned model generates more coherent, structured responses compared to the vanilla model, which often provides irrelevant answers. The LoRA model tends to follow the prompt more accurately, offering step-by-step guidance (e.g., creating a marketing plan) and correct interpretations (e.g., the Italian translation of “house”). However, it occasionally adds extraneous or overly detailed information, making responses verbose. The vanilla model, while shorter, often lacks relevance and logical flow, producing confusing responses making it less reliable. Overall, the LoRA model delivers more usable and intelligible answers. This improvement would likely be more significant with fine-tuning larger LLMs.